# Data Wrangling Challenge
### Pull and manipulate the API data

The point of this exercise is to try data enrichment with data from external APIs. We are going to take data about car crashes in Monroe County, Indiana from 2003 to 2015 and try to figure out the weather during the accident and how many bars there are in the area. We will work with Following APIs during this challenge:

- [Yelp](https://www.yelp.com/developers/documentation/v3/get_started) - to put the number of bars and restaurants in the area!
- [World Weather Online API](https://www.worldweatheronline.com/developer/api/historical-weather-api.aspx) as **Stretch work**

We will try to find correlations between the severity of crash and weather/number of bars in the area. To indicate the severity of a crash, we will use column `Injury Type`.

## Data

The data for this exercise can be found [here](https://drive.google.com/file/d/1_KF9oIJV8cB8i3ngA4JPOLWIE_ETE6CJ/view?usp=sharing).

Just run the cells below to get your data ready. Little help from us.


In [5]:
import pandas as pd
import requests

In [34]:
data = pd.read_csv("data/monroe-county-crash-data2003-to-2015.csv", encoding="unicode_escape")
# ========================
# preparing data
data.dropna(subset=['Latitude', 'Longitude'], inplace=True)
# creation of variable with lon and lat together
data['ll'] = data['Latitude'].astype(str) + ',' + data['Longitude'].astype(str)
data = data[data['ll'] != '0.0,0.0']
print(data.shape)
data.head()

(49005, 13)


,Master Record Number,Year,Month,Day,Weekend?,Hour,Collision Type,Injury Type,Primary Factor,Reported_Location,Latitude,Longitude,ll
0,902363382,2015,1,5,Weekday,0.0,2-Car,No injury/unknown,OTHER (DRIVER) - EXPLAIN IN NARRATIVE,1ST & FESS,39.159207,-86.525874,"39.15920668,-86.52587356"
1,902364268,2015,1,6,Weekday,1500.0,2-Car,No injury/unknown,FOLLOWING TOO CLOSELY,2ND & COLLEGE,39.161440,-86.534848,"39.16144,-86.534848"
2,902364412,2015,1,6,Weekend,2300.0,2-Car,Non-incapacitating,DISREGARD SIGNAL/REG SIGN,BASSWOOD & BLOOMFIELD,39.149780,-86.568890,"39.14978027,-86.56889006"
3,902364551,2015,1,7,Weekend,900.0,2-Car,Non-incapacitating,FAILURE TO YIELD RIGHT OF WAY,GATES & JACOBS,39.165655,-86.575956,"39.165655,-86.57595635"
4,902364615,2015,1,7,Weekend,1100.0,2-Car,No injury/unknown,FAILURE TO YIELD RIGHT OF WAY,W 3RD,39.164848,-86.579625,"39.164848,-86.57962482"


In [45]:
data['Injury Type'].value_counts()

No injury/unknown     37467
Non-incapacitating    10427
Incapacitating         1003
Fatal                   108
Name: Injury Type, dtype: int64

In [46]:
data['severity'] = 0
data.loc[data['Injury Type'] == 'Non-incapacitating', 'severity'] = 1
data.loc[data['Injury Type'] == 'Incapacitating', 'severity'] = 2
data.loc[data['Injury Type'] == 'Fatal', 'severity'] = 3

In [58]:
data['date'] = (data.Year.astype(str) + 
                "-"
                data.Month.astype(str).str.zfill(2) + 
                "-"
                data.Day.astype(str).str.zfill(2))

SyntaxError: invalid syntax (<ipython-input-58-ecfc233bf5d2>, line 3)

In [ ]:
data.head()

# Yelp API

Yelp API documentation is [here](https://www.yelp.com/developers/documentation/v3/get_started)

1. Get Your API Key
2. For each crash, use endpoint `/businesses/search` to pull number of nightlife bar and other entertainments in the radius of 10km around the crash
3. Find a relationship (if there is any) between number of bars in the area and severity of the crash.

HINTs: 
- from the beginning, start pulling number of bars for smaller sample of crashes (500 or so) only
- **list of categories** for bars and nightlife can to be found in the [Yelp API documentation](https://www.yelp.com/developers/documentation/v3/all_category_list)

In [47]:
#set the keys
apikey = ""

headers = {
        'Authorization': f'Bearer {apikey}'
    }

In [55]:
test_data = data.head(1000)

In [56]:
lats = test_data.Latitude.tolist()
lons = test_data.Longitude.tolist()

In [57]:
# # getting restauraYears in the neighborhood
totals = []
for i in range(len(lats)):
# for i in range(5):
    url_params = {
            'latitude': str(lats[i]),
            'longitude': str(lons[i]),
            'categories': 'nightlife',
            'radius': 1000
        }

    url = 'https://api.yelp.com/v3/businesses/search'

    resp = requests.get(url=url, headers=headers, params=url_params)
    res = resp.json()
    totals.append(res['total'])

In [52]:
test_data['no_bars'] = totals

/Users/jurajkapasny/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [53]:
test_data[['severity','no_bars']].corr()

,severity,no_bars
severity,1.000000,-0.092392
no_bars,-0.092392,1.000000


# Stretch

## World Weather Online API

World Weather Online API is [here](https://www.worldweatheronline.com/developer/api/historical-weather-api.aspx)

1. Sign up for FREE api key if you haven't done that before (it's free for **60 days**).
2. For each crush, get the weather for the location and date.
3. Find a relationship between the weather and severity of the crash.


In [ ]:
import requests
import time
api_key = ''